# Deep Learning

This notebook is to handele the validation part only

In [1]:
import cPickle as pickle
import numpy as np
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import math

### Hyperparameter

In [2]:
basket_static ='/mnt/basket/processingData/data/notMNIST'
basket_volatile = "/mnt/basket/processingData/volatile/notMNIST"
raw_data_pickle_filename = "notMNIST.pickle"
training_results_pickle_filename = "2_fullyconnected_separeated__training_results.pickle"

## Load the data

In [3]:
# Data from notebook 01_NormalizeImage.ipynb
pickle_file =  os.path.join(basket_volatile, raw_data_pickle_filename)

with open(pickle_file, 'rb') as f:
  read = pickle.load(f)
  valid_dataset = read['valid_dataset']
  valid_labels = read['valid_labels']
  del read  # hint to help gc free up memory
  print 'Validation set', valid_dataset.shape, valid_labels.shape


# Data from notebook 02_Training.ipynb
pickle_file = os.path.join(basket_volatile,training_results_pickle_filename)

with open(pickle_file, 'rb') as f:
  read = pickle.load(f)
  train_weights_hidden = read['weights_hidden_1']
  train_biases_hidden = read['biases_hidden_1']
  train_weights_linear = read['weights_linear']
  train_biases_linear = read['biases_linear']
  del read  # hint to help gc free up memory

Validation set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix
- labels as float 1-hot encodings

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
print 'Valid dataset', valid_dataset.shape, valid_labels.shape

Valid dataset (10000, 784) (10000, 10)


### Utility function

In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## S.D.G. with one hidden layer

### Testing

##### Define the graph

In [6]:
batch_size = 128
learning_rate = 0.5
graph = tf.Graph()

with graph.as_default():
    
  #Input data. For the training data.
  tf_valid_dataset = tf.constant(valid_dataset) #(18724,784)

    
  #Define HyperParameters (NN structuring constants)
  inputLayerSize = image_size * image_size # inputLayerSize is always equal to width of X-ndarray (200000,784)
  hiddenLayerSize = 1024
  outputLayerSize = 10
    
  # Variables:
  weights_hidden_1 = tf.constant(train_weights_hidden) #(784,1024)
  biases_hidden_1 = tf.constant(train_biases_hidden) #(1024)
  ## linear layer
  weights_linear = tf.constant(train_weights_linear) #(1024,10)
  biases_linear = tf.constant(train_biases_linear) #(10)


  # Hidden layer (RELU)
  valid_hiddenLayer = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_hidden_1) + biases_hidden_1) #(128,1024)

  # linear layer
  valid_logits = tf.matmul(valid_hiddenLayer, weights_linear) + biases_linear  # tune w2,h2

 
  # Predictions for the training, validation, and test data.
  valid_prediction = tf.nn.softmax(valid_logits)

##### Run the graph

In [7]:
num_steps = 3001
plt.clf()

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)

Validation accuracy: 82.2%
